In [2]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import tokenizer_from_json
import tensorflow as tf
import numpy as np
import json
import re
import string

# Load the saved tokenizers and models
# Load tokenizer from JSON file
with open('/content/drive/MyDrive/Sem_8-G21/model/input_tokenizer.json', 'r') as f:
    input_tokenizer = tokenizer_from_json(json.dumps(json.load(f)))

# Similarly, load the Gujarati tokenizer
with open('/content/drive/MyDrive/Sem_8-G21/model/output_tokenizer.json', 'r') as f:
    output_tokenizer = tokenizer_from_json(json.dumps(json.load(f)))

    
normal_model = tf.keras.models.load_model('/content/drive/MyDrive/Sem_8-G21/model/eng-to-gujarati.h5')
encoder_model = tf.keras.models.load_model('/content/drive/MyDrive/Sem_8-G21/model/Encoder-Model.h5')
decoder_model = tf.keras.models.load_model('/content/drive/MyDrive/Sem_8-G21/model/Decoder-Model.h5')

normal_model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
encoder_model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
decoder_model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

def decode_sequence(input_sentence):
    # Encode the input as state vectors.
    enc_output, enc_hidden = encoder_model.predict(input_tokenizer.texts_to_sequences([input_sentence]))

    # Generate an empty target sequence of length 1.
    target_sequence = np.zeros((1, 1))

    # Populate the first character of target sequence with the start character.
    target_sequence[0, 0] = output_tokenizer.word_index['start']

    # Initialize the decoder state with the encoder hidden state.
    dec_hidden = enc_hidden
    stop_condition = False
    guj_sentence = ''
    
    while not stop_condition:
        dec_output, dec_hidden, attention_weights = decoder_model.predict([target_sequence, dec_hidden, enc_output])

        # Sample a token from the output distribution
        sampled_token_index = np.argmax(dec_output[0, -1, :])
        
        if sampled_token_index not in output_tokenizer.index_word:
            break
            
        sampled_token = output_tokenizer.index_word[sampled_token_index]

        # Append the sampled token to the target sequence
        guj_sentence += ' ' + sampled_token

        # Exit condition: either hit max length or find stop character.
        if (sampled_token == 'end' or len(guj_sentence) > 20):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_sequence = np.zeros((1, 1))
        target_sequence[0, 0] = sampled_token_index

    return guj_sentence.strip()


# Test the model with some example sentences
input_sentence = 'my name is harsh'
guj_sentence = decode_sequence(input_sentence)
print(input_sentence + ' => ' + guj_sentence)

input_sentence = 'how are you'
guj_sentence = decode_sequence(input_sentence)
print(input_sentence + ' => ' + guj_sentence)

input_sentence = 'how is this'
guj_sentence = decode_sequence(input_sentence)
print(input_sentence + ' => ' + guj_sentence)

# # Save the model as a TensorFlow SavedModel
# tf.saved_model.save(normal_model, 'normal_model_saved_model')
# tf.saved_model.save(encoder_model, 'encode_model_saved_model')
# tf.saved_model.save(decoder_model, 'decode_model_saved_model')


1/1 [==============================] - 0s 31ms/step
my name is harsh => કોઈનેય
1/1 [==============================] - 0s 30ms/step
how are you => મનની ચંદ્રાયાન એમએમયુ
1/1 [==============================] - 0s 30ms/step
how is this => ટ્રેનના ઓક્ટોબરને ધર્યા
